### CLUSTERING NON SEPARABLE DATA WITH GIBBS SAMPLING FOR GAUSSIAN MIXTURE MODELS

In [ ]:
# Packages

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import dirichlet, multivariate_normal, invwishart
from matplotlib.patches import Ellipse
from itertools import product
from matplotlib.colors import LinearSegmentedColormap, Normalize
from matplotlib.cm import ScalarMappable
import matplotlib.colors as mcolors


In [ ]:
# Function for normalizing data

def normalize_data(data):
    mean = np.mean(data, axis=0)
    std = np.std(data, axis=0)
    normalized_data = (data - mean) / std
    return normalized_data, mean, std
    

### NHANES DATASET

In [ ]:

df = pd.read_csv('NHANES_adults_data_preproc.csv')

# Select the columns for BMI and SBP
df_bmi_sbp = df[['bmi', 'sbp']]

# Check for missing values in the selected columns
missing_values = df_bmi_sbp.isnull().sum()
print('-' * 30)
print("Missing values before removal:")
print(missing_values)
print('-' * 30)

# Remove rows with missing values
df_bmi_sbp_clean = df_bmi_sbp.dropna()

# Normalize the clean data
data_US = df_bmi_sbp_clean.values
data_norm_US_full, mean_US, std_US = normalize_data(data_US)

# Convert the normalized data back to a DataFrame
df_bmi_sbp_norm = pd.DataFrame(data_norm_US_full, columns=['bmi', 'sbp'])

# Take a random sample of 300 from the normalized data
data_norm_US = df_bmi_sbp_norm.sample(n=300, replace=False, random_state=5)

# Plot the scatter plot of normalized BMI vs SBP
plt.figure(figsize=(8, 5))
plt.scatter(data_norm_US['bmi'], data_norm_US['sbp'], alpha=0.5, color='purple')
plt.title('Scatter Plot of Normalized BMI vs SBP (NHANES Dataset Subset)')
plt.xlabel('Normalized BMI')
plt.ylabel('Normalized SBP (Systolic Blood Pressure)')
plt.grid(True)
plt.show()

data_norm_US = data_norm_US.values

### Simulated data

In [ ]:
def create_data(seed=None, means=[(-2, -2), (0, 0), (2, 2)], covariances=[np.eye(2), np.eye(2), np.eye(2)], n_samples=100):
    if seed is not None:
        np.random.seed(seed)
        
    data = np.vstack([
        np.random.multivariate_normal(means[0], covariances[0], n_samples),
        np.random.multivariate_normal(means[1], covariances[1], n_samples),
        np.random.multivariate_normal(means[2], covariances[2], n_samples)
    ])
    
    true_labels = np.hstack([
        np.zeros(n_samples),
        np.ones(n_samples),
        np.full(n_samples, 2)
    ])
    
    return data, true_labels



data, true_labels = create_data(1)
data_norm, mean, std = normalize_data(data)


### Functions for sampling

In [ ]:
# Initialization based on random subset

def initialize_parameters_random_subset(data, K, s=None):
    N, D = data.shape
    
    if s is not None:
        np.random.seed(s)
    
    # Step 1: Select K random data points as initial means
    random_indices = np.random.choice(N, K, replace=False)
    mu = data[random_indices]
    
    # Step 2: Assign initial cluster labels based on closest mean
    z = np.zeros(N, dtype=int)
    for i in range(N):
        distances = np.linalg.norm(data[i] - mu, axis=1)
        z[i] = np.argmin(distances)
    
    # Step 3: Initialize covariances
    Sigma = np.zeros((K, D, D))
    for k in range(K):
        cluster_data = data[z == k]
        if len(cluster_data) > 1:
            Sigma[k] = np.cov(cluster_data, rowvar=False)
        else:
            Sigma[k] = np.eye(D)  # if a cluster has only one point, use identity matrix as covariance
    
    # Step 4: Initialize mixing coefficients
    pi = np.zeros(K)
    for k in range(K):
        pi[k] = np.sum(z == k) / N
    
    return z, pi, mu, Sigma


In [ ]:
def sample_z(data, pi, mu, Sigma, K):
    N = data.shape[0]
    z = np.zeros(N, dtype=int)
    probabilities = np.zeros((N, K))
    
    for i in range(N):
        probs = np.array([pi[k] * multivariate_normal.pdf(data[i], mean=mu[k], cov=Sigma[k]) for k in range(K)])
        probs /= np.sum(probs)
        z[i] = np.argmax(np.random.multinomial(1, probs))
        probabilities[i] = probs

    return z, probabilities

# Computes the probability of each data point belonging to each cluster and samples a cluster assignment based on these probabilities



def sample_pi(z, K, alpha):
    counts = np.array([np.sum(z == k) for k in range(K)])
    # counts = np.bincount(z, minlength=len(alpha))
    pi = dirichlet.rvs(alpha + counts, size=1)
    return pi[0]



def sample_mu(data, z, K, V0, m0, Sigma):
    N, D = data.shape
    mu = np.zeros((K, D))
    
    for k in range(K):
        Nk = np.sum(z == k)
        if Nk > 0:
            xk = np.mean(data[z == k], axis=0)
            # Update V_k and m_k
            V_k_inv = np.linalg.inv(V0) + Nk * np.linalg.inv(Sigma[k])
            V_k = np.linalg.inv(V_k_inv)
            m_k = V_k @ (np.linalg.inv(Sigma[k]) @ (Nk * xk) + np.linalg.inv(V0) @ m0)
            
            # Sample mu_k
            mu[k] = multivariate_normal.rvs(mean=m_k, cov=V_k)
        else:
            # Sample from the prior distribution
            mu[k] = multivariate_normal.rvs(mean=m0, cov=V0)
    
    return mu


def sample_Sigma(data, z, K, mu, S0, nu0):
    N, D = data.shape
    Sigma = np.zeros((K, D, D))
    
    for k in range(K):
        Nk = np.sum(z == k)
        
        # Update S_k and nu_k
        if Nk > 0:
            S_k = S0 + np.sum([np.outer(data[i] - mu[k], data[i] - mu[k]) for i in range(N) if z[i] == k], axis=0)
        else:
            S_k = S0  # Sample from prior if Nk == 0
            
        nu_k = nu0 + Nk
        
        # Sample Sigma_k
        Sigma[k] = invwishart.rvs(df=nu_k, scale=S_k)

    return Sigma


# Log likelihood 

def compute_log_likelihood(data, pi, mu, Sigma, K):
    N = data.shape[0]
    log_likelihood = 0
    for i in range(N):
        likelihood_i = 0
        for k in range(K):
            likelihood_i += pi[k] * multivariate_normal.pdf(data[i], mean=mu[k], cov=Sigma[k])
        log_likelihood += np.log(likelihood_i)
    return log_likelihood



def gibbs_sampling_gmm(data, K, alpha, m0, V0, S0, nu0, initialization_function, num_iterations, s=None):
    N, D = data.shape
    
    # Initialize variables
    z, pi, mu, Sigma = initialization_function(data, K, s)

    
    samples = {'z': [], 'probabilities': [],'pi': [],'mu': [],'Sigma': [], 'log_likelihood': []}
    
    for iteration in range(num_iterations):
        # Sample z
        z, probabilities = sample_z(data, pi, mu, Sigma, K)
        
        # Sample pi
        pi = sample_pi(z, K, alpha)
        
        # Sample mu
        mu = sample_mu(data, z, K, V0, m0, Sigma)
        
        # Sample Sigma
        Sigma = sample_Sigma(data, z, K, mu, S0, nu0)

        log_likelihood = compute_log_likelihood(data, pi, mu, Sigma, K)
        samples['log_likelihood'].append(log_likelihood)
        # Append current samples to the list
        samples['z'].append(z.copy())
        samples['probabilities'].append(probabilities.copy())
        samples['pi'].append(pi.copy())
        samples['mu'].append(mu.copy())
        samples['Sigma'].append(Sigma.copy())
    
    return samples


### Functions for plotting

In [ ]:

def plot_trace_plots(samples, num_iterations, K, S0, nu0):
    fig, axes = plt.subplots(3, 1, figsize=(8, 14))

    # Plot trace of pi
    for k in range(K):
        axes[0].plot([samples['pi'][i][k] for i in range(num_iterations)], label=f'pi_{k}')
    axes[0].set_title(f'Trace plot of pi with S0={S0}, nu0={nu0}')
    axes[0].legend()

    # Plot trace of mu
    for k in range(K):
        for d in range(samples['mu'][0].shape[1]):
            axes[1].plot([samples['mu'][i][k, d] for i in range(num_iterations)], label=f'mu_{k}_{d}')
    axes[1].set_title('Trace plot of mu with S0={S0}, nu0={nu0}')
    axes[1].legend()

    # Plot trace of Sigma (diagonal elements only)
    for k in range(K):
        for d in range(samples['Sigma'][0].shape[1]):
            axes[2].plot([samples['Sigma'][i][k, d, d] for i in range(num_iterations)], label=f'Sigma_{k}_{d}_{d}')
    axes[2].set_title('Trace plot of Sigma (diagonal elements) with S0={S0}, nu0={nu0}')
    axes[2].legend()

    plt.tight_layout()
    plt.show()


def plot_ellipse(ax, mu, cov, color):
    """
    Plot an ellipse representing the covariance matrix.
    """
    # Eigenvalues and eigenvectors of the covariance matrix
    eigenvalues, eigenvectors = np.linalg.eigh(cov)
    
    # Order the eigenvalues and eigenvectors by descending eigenvalues
    order = eigenvalues.argsort()[::-1]
    eigenvalues = eigenvalues[order]
    eigenvectors = eigenvectors[:, order]
    
    # Angle of the ellipse
    angle = np.degrees(np.arctan2(*eigenvectors[:, 0][::-1]))
    
    # Width and height of the ellipse (2 standard deviations)
    width, height = 2 * 2 * np.sqrt(eigenvalues)
    
    # Plot the ellipse
    ellipse = Ellipse(xy=mu, width=width, height=height, angle=angle, edgecolor=color, facecolor='none')
    ax.add_patch(ellipse)



def plot_log_likelihood(samples, title):
    plt.figure(figsize=(9, 5))
    plt.plot(samples['log_likelihood'])
    plt.title(title, fontsize=16)
    plt.xlabel('Iteration', fontsize=13)
    plt.ylabel('Log-Likelihood', fontsize=13)
    plt.grid(True)
    plt.show()


In [ ]:
def create_fixed_palette(selected_indices, table='tab10'):
    colormap = plt.get_cmap(table)
    fixed_palette = np.array([colormap(i)[:3] for i in selected_indices])
    return fixed_palette

def probabilities_to_rgb(probabilities, fixed_palette):
    # probabilities: Array of shape (N, K) representing probabilities for N data points and K clusters
    # fixed_palette: Array of shape (K, 3) representing RGB colors for K clusters
    rgb_colors = probabilities @ fixed_palette
    return rgb_colors


In [ ]:
def plot_iterations_rgb(samples, iterations, data, K, fixed_palette, title, x_axis, y_axis):
    fig, ax = plt.subplots(2, 2, figsize=(12, 8))
    fig.suptitle(title, fontsize=17)
    
    for i, iteration in enumerate(iterations):
        z = samples['z'][iteration]
        probabilities = samples['probabilities'][iteration]
        mu = samples['mu'][iteration]
        Sigma = samples['Sigma'][iteration]
        rgb_colors = probabilities_to_rgb(probabilities, fixed_palette)
        
        row, col = divmod(i, 2)
        scatter = ax[row, col].scatter(data[:, 0], data[:, 1], c=rgb_colors, marker='o')
        ax[row, col].set_title(f'Clustered Data at Iteration {iteration}', fontsize=15)
        ax[row, col].set_xlabel(x_axis, fontsize=12)
        ax[row, col].set_ylabel(y_axis, fontsize=12)
        
        for k in range(K):
            ax[row, col].scatter(mu[k, 0], mu[k, 1], c='black', s=200, marker='x', linewidths=4.5)
            ax[row, col].scatter(mu[k, 0], mu[k, 1], c=[fixed_palette[k]], s=120, marker='x', linewidths=2.0)
            plot_ellipse(ax[row, col], mu[k], Sigma[k], color=fixed_palette[k])
    
    # Add continuous color bar 
    norm = Normalize(vmin=0, vmax=1)
    cbar = plt.colorbar(ScalarMappable(norm=norm, cmap=LinearSegmentedColormap.from_list("custom_cmap", fixed_palette)), ax=ax[0, 1], orientation='vertical')
    cbar.set_ticks([])  
    cbar.set_label('Probability of Cluster Membership', fontsize=14)
  
    legend_handles = []
    for k in range(K):
        handle = plt.Line2D([0], [0], marker='x', color=fixed_palette[k], markersize=10, linestyle='None', label=f'Cluster {k + 1} Mean')
        legend_handles.append(handle)
    
    ax[1, 1].legend(handles=legend_handles, loc='upper right', prop={'size': 10})
    
    plt.tight_layout()
    plt.show()


## IMPLEMENTATION OF GIBBS SAMPLER FOR GMM

In [ ]:
# Parameters for the Gibbs sampling
K = 3
alpha = np.ones(K)
m0 = np.zeros(2)
V0 = np.eye(2)
nu0 = 4
num_iterations = 3000

selected_indices = [0, 2, 1] 
fixed_palette = create_fixed_palette(selected_indices)

### For Simulated dataset

In [ ]:
# Plot simulated data with true labels

plt.figure(figsize=(8, 5))
corrected_palette = fixed_palette.copy()
corrected_palette[1], corrected_palette[2] = fixed_palette[2], fixed_palette[1]
cmap = mcolors.ListedColormap(corrected_palette)
norm = mcolors.BoundaryNorm(boundaries=np.arange(len(corrected_palette) + 1) - 0.5, ncolors=len(corrected_palette))
scatter = plt.scatter(data_norm[:, 0], data_norm[:, 1], c=true_labels, cmap=cmap, norm=norm, marker='o')
plt.title('Scatter Plot of Simulated Data with True Labels', fontsize=15)
plt.xlabel('Feature 1', fontsize=13)
plt.ylabel('Feature 2', fontsize=13)

cbar_palette = [corrected_palette[0], corrected_palette[2], corrected_palette[1]]
cbar_cmap = mcolors.ListedColormap(cbar_palette)
cbar_norm = mcolors.BoundaryNorm(boundaries=np.arange(len(cbar_palette) + 1) - 0.5, ncolors=len(cbar_palette))

cbar = plt.colorbar(ScalarMappable(cmap=cbar_cmap, norm=cbar_norm), ax=scatter.axes, ticks=[0, 1, 2])
cbar.ax.set_yticklabels(['Cluster 1', 'Cluster 2', 'Cluster 3'], fontsize=11)  
cbar.set_label('True Cluster Labels', fontsize=13)

plt.grid(True)
plt.show()


In [ ]:
S0 =  np.cov(data_norm, rowvar=False)
# Run Gibbs sampling
samples_gibbs = gibbs_sampling_gmm(data_norm, K, alpha, m0, V0, S0, nu0, initialize_parameters_random_subset, num_iterations=num_iterations, s=1)

# Get the final clustering result
z_gibbs = samples_gibbs['z'][-1]
final_mu_gibbs = samples_gibbs['mu'][-1]
final_Sigma_gibbs = samples_gibbs['Sigma'][-1]


In [ ]:
plot_log_likelihood(samples_gibbs, 'Log-Likelihood vs Iterations - Simulated Data')
plot_iterations_rgb(samples_gibbs, [450, 1000, 2400, 2999], data_norm, K, fixed_palette, 'Clustered Data Across Iterations - Simulated Data', 'Feature 1', 'Feature 2')


### For NHANES dataset

In [ ]:
S0 =  np.cov(data_norm_US, rowvar=False)

samples_gibbs_US = gibbs_sampling_gmm(data_norm_US, K, alpha, m0, V0, S0, nu0, initialize_parameters_random_subset, num_iterations=num_iterations, s=7)

z_gibbs_US = samples_gibbs_US['z'][-1]
final_mu_gibbs_US = samples_gibbs_US['mu'][-1]
final_Sigma_gibbs_US = samples_gibbs_US['Sigma'][-1]

In [ ]:
plot_log_likelihood(samples_gibbs_US, 'Log-Likelihood vs Iterations - NHANES Dataset')
plot_iterations_rgb(samples_gibbs_US, [350, 900, 1200, 2990], data_norm_US, K, fixed_palette, 'Clustered Data Across Iterations - NHANES Dataset', 'BMI', 'SBP (Systolic Blood Pressure)')


## HYPERPARAMETERS

In [ ]:
def plot_clusters_posterior(data, samples, K, fixed_palette, S0, nu0, title, x_axis, y_axis, burn_in=100):
    mu_samples = np.array(samples['mu'][burn_in:])
    Sigma_samples = np.array(samples['Sigma'][burn_in:])
    pi_samples = np.array(samples['pi'][burn_in:])
    posterior_mu = np.mean(mu_samples, axis=0)
    posterior_Sigma = np.mean(Sigma_samples, axis=0)
    posterior_pi = np.mean(pi_samples, axis=0)
    
    # Print the posterior pi with cluster labels
    for k in range(K):
        print(f"Posterior pi for cluster {k+1}: {posterior_pi[k]}")

    # Calculate and print the minimum distance between cluster means
    min_distance = np.inf
    for i in range(K):
        for j in range(i+1, K):
            distance = np.linalg.norm(posterior_mu[i] - posterior_mu[j])
            min_distance = min(min_distance, distance)
    print(f"Minimum distance between cluster means: {min_distance:.3f}")

    _, probabilities_posterior = sample_z(data, posterior_pi, posterior_mu, posterior_Sigma, K)
    rgb_colors_posterior = probabilities_to_rgb(probabilities_posterior, fixed_palette)
    
    plt.figure(figsize=(10, 6))
    plt.scatter(data[:, 0], data[:, 1], c=rgb_colors_posterior, marker='o')

    for k in range(K):
        plt.scatter(posterior_mu[k, 0], posterior_mu[k, 1], c='black', marker='x', s=250, linewidths=4.5)
        plt.scatter(posterior_mu[k, 0], posterior_mu[k, 1], c=[fixed_palette[k]], marker='x', s=120, linewidths=2.0)
        plot_ellipse(plt.gca(), posterior_mu[k], posterior_Sigma[k], color=fixed_palette[k])

    for k in range(K):
        plt.scatter([], [], c=[fixed_palette[k]], marker='x', s=120, label=f'Cluster {k+1} Mean')

    plt.title(
    f'Clustered {title} Data: Posterior Mean of $\mu$ '
    f'($\\nu_0$={nu0}, $S_0$=[[{int(S0[0,0])}, {int(S0[0,1])}], '
    f'[{int(S0[1,0])}, {int(S0[1,1])}]])', fontsize=15)

    plt.xlabel(x_axis, fontsize=13)
    plt.ylabel(y_axis, fontsize=13)
    plt.legend(fontsize=12)
    plt.show()


### Simulated dataset

In [ ]:
S0 = np.eye(2)
nu0 = 4
num_iterations = 3000

# Run the Gibbs sampling
samples_1 = gibbs_sampling_gmm(data_norm, K, alpha, m0, V0, S0, nu0, initialize_parameters_random_subset, num_iterations)

In [ ]:
plot_clusters_posterior(data_norm, samples_1, K, fixed_palette, S0, nu0, 'Simulated', 'Feature 1', 'Feature 2', burn_in=100)

In [ ]:
S0 = np.eye(2)
nu0 = 50

samples_2 = gibbs_sampling_gmm(data_norm, K, alpha, m0, V0, S0, nu0, initialize_parameters_random_subset, num_iterations=3000)

In [ ]:
plot_clusters_posterior(data_norm, samples_2, K, fixed_palette, S0, nu0, 'Simulated', 'Feature 1', 'Feature 2', burn_in=100)

In [ ]:
S0 = 6*np.eye(2)
nu0 = 4

# Run the Gibbs sampling 
samples_3 = gibbs_sampling_gmm(data_norm, K, alpha, m0, V0, S0, nu0, initialize_parameters_random_subset, num_iterations)

In [ ]:
plot_clusters_posterior(data_norm, samples_3, K, fixed_palette, S0, nu0, 'Simulated', 'Feature 1', 'Feature 2', burn_in=200)

In [ ]:
S0 = 6*np.eye(2)
nu0 = 50

samples_4 = gibbs_sampling_gmm(data_norm, K, alpha, m0, V0, S0, nu0, initialize_parameters_random_subset, num_iterations)

In [ ]:
plot_clusters_posterior(data_norm, samples_4, K, fixed_palette, S0, nu0, 'Simulated', 'Feature 1', 'Feature 2', burn_in=100)

### NHANES Dataset

In [ ]:
S0 = np.eye(2)
nu0 = 4

samples_1_US = gibbs_sampling_gmm(data_norm_US, K, alpha, m0, V0, S0, nu0, initialize_parameters_random_subset, num_iterations)

In [ ]:
plot_clusters_posterior(data_norm_US, samples_1_US, K, fixed_palette, S0, nu0, 'NHANES', 'BMI', 'SBP', burn_in=100)

In [ ]:
S0 = 6 * np.eye(2)
nu0 = 50

# Run the Gibbs sampling and store the samples
samples_2_US = gibbs_sampling_gmm(data_norm_US, K, alpha, m0, V0, S0, nu0, initialize_parameters_random_subset, num_iterations)

In [ ]:
plot_clusters_posterior(data_norm_US, samples_2_US, K, fixed_palette, S0, nu0, 'NHANES', 'BMI', 'SBP', burn_in=150)

## Incorporating Constraints

In [ ]:
def plot_two_iterations(samples, iterations, data, K, fixed_palette, S0, nu0, title, x_axis, y_axis):
    fig, ax = plt.subplots(1, 2, figsize=(12, 6))  
    
    for i, iteration in enumerate(iterations):
        z = samples['z'][iteration]
        probabilities = samples['probabilities'][iteration]
        mu = samples['mu'][iteration]
        Sigma = samples['Sigma'][iteration]
        rgb_colors = probabilities_to_rgb(probabilities, fixed_palette)
        
        ax[i].scatter(data[:, 0], data[:, 1], c=rgb_colors, marker='o')
        ax[i].set_title(f'Clustered Data at Iteration {iteration}', fontsize=15)
        ax[i].set_xlabel(x_axis, fontsize=11)
        ax[i].set_ylabel(y_axis, fontsize=11)
        
        # Plot the cluster means and ellipses
        for k in range(K):
            ax[i].scatter(mu[k, 0], mu[k, 1], c='black', s=250, marker='x', linewidths=4.5)
            ax[i].scatter(mu[k, 0], mu[k, 1], c=[fixed_palette[k]], s=120, marker='x', linewidths=2.0)
            plot_ellipse(ax[i], mu[k], Sigma[k], color=fixed_palette[k])
        
        # Compute the minimum distance between cluster means
        min_distance = np.inf
        for m in range(K):
            for n in range(m + 1, K):
                distance = np.linalg.norm(mu[m] - mu[n])
                min_distance = min(min_distance, distance)
        
        print(f'Minimum distance between cluster means at iteration {iteration}: {min_distance:.3f}')
        
        if i == 0:
            legend_handles = []
            for k in range(K):
                handle = plt.Line2D([0], [0], marker='x', color=fixed_palette[k], markersize=10, linestyle='None', label=f'Cluster {k + 1} Mean')
                legend_handles.append(handle)
            ax[i].legend(handles=legend_handles, loc='upper right', prop={'size': 12})

    fig.suptitle(title, fontsize=18)
    plt.tight_layout(rect=[0, 0, 1, 1]) 
     
    plt.show()



### Constraint h1

In [ ]:
# Constraint function h1
def h1(mu):
    K = len(mu)
    min_distance = float('inf')
    for k in range(K):
        for k_prime in range(k + 1, K):
            distance = np.linalg.norm(mu[k] - mu[k_prime])
            if distance < min_distance:
                min_distance = distance
    return min_distance


In [ ]:
def sample_mu_metropolis_h1(data, z, K, V0, m0, Sigma, mu_current, sigma_proposal, h):
    N, D = data.shape
    mu_proposed = np.zeros((K, D))
    
    # Proposal distr (multivariate normal distribution)
    for k in range(K):
        mu_proposed[k] = multivariate_normal.rvs(mean=mu_current[k], cov=sigma_proposal * np.eye(D))
    # Compute the log-posterior for the current and proposed means
    log_posterior_current = 0
    log_posterior_proposed = 0
    for k in range(K):
        # Log-prior
        log_posterior_current += multivariate_normal.logpdf(mu_current[k], mean=m0, cov=V0)
        log_posterior_proposed += multivariate_normal.logpdf(mu_proposed[k], mean=m0, cov=V0)
        # Log-likelihood
        for i in range(N):
            if z[i] == k:
                log_posterior_current += multivariate_normal.logpdf(data[i], mean=mu_current[k], cov=Sigma[k])
                log_posterior_proposed +=  multivariate_normal.logpdf(data[i], mean=mu_proposed[k], cov=Sigma[k])
    # Add separation constraint term
    h_current = h(mu_current)
    h_proposed = h(mu_proposed)

    log_posterior_current += np.log(h_current) 
    log_posterior_proposed += np.log(h_proposed) 
    # Compute the acceptance ratio
    acceptance_ratio = log_posterior_proposed - log_posterior_current
    # Accept or reject the proposal
    if  np.log(np.random.uniform(0, 1)) <= acceptance_ratio:
        mu = mu_proposed
    else:
        mu = mu_current
    
    return mu


In [ ]:
def gibbs_sampling_gmm_h1(data, K, alpha, m0, V0, S0, nu0, initialization_function, sigma_proposal, num_iterations, h, s=None):
    N, D = data.shape
    
    # Initialize variables
    z, pi, mu, Sigma = initialization_function(data, K, s)

    samples = {'z': [], 'probabilities': [], 'pi': [], 'mu': [], 'Sigma': [], 'log_likelihood': []}
    
    for iteration in range(num_iterations):
        # Sample z
        z, probabilities = sample_z(data, pi, mu, Sigma, K)
        
        # Sample pi
        pi = sample_pi(z, K, alpha)
        
        # Sample mu using Metropolis within Gibbs
        mu = sample_mu_metropolis_h1(data, z, K, V0, m0, Sigma, mu, sigma_proposal, h)
        
        # Sample Sigma
        Sigma = sample_Sigma(data, z, K, mu, S0, nu0)

        # Store log-likelihood
        log_likelihood = compute_log_likelihood(data, pi, mu, Sigma, K)
        samples['log_likelihood'].append(log_likelihood)

        # Append current samples to the list
        samples['z'].append(z.copy())
        samples['probabilities'].append(probabilities.copy())
        samples['pi'].append(pi.copy())
        samples['mu'].append(mu.copy())
        samples['Sigma'].append(Sigma.copy())
    
    return samples

In [ ]:
def plot_clusters_posterior_h(data, samples, K, fixed_palette, S0, nu0, title, x_axis, y_axis, burn_in=100):
    mu_samples = np.array(samples['mu'][burn_in:])
    Sigma_samples = np.array(samples['Sigma'][burn_in:])
    pi_samples = np.array(samples['pi'][burn_in:])
    posterior_mu = np.mean(mu_samples, axis=0)
    posterior_Sigma = np.mean(Sigma_samples, axis=0)
    posterior_pi = np.mean(pi_samples, axis=0)
    
    # Print the posterior pi with cluster labels
    for k in range(K):
        print(f"Posterior pi for cluster {k+1}: {posterior_pi[k]}")

    # Calculate and print the minimum distance between cluster means
    min_distance = np.inf
    for i in range(K):
        for j in range(i+1, K):
            distance = np.linalg.norm(posterior_mu[i] - posterior_mu[j])
            min_distance = min(min_distance, distance)
    print(f"Minimum distance between cluster means: {min_distance:.3f}")

    _, probabilities_posterior = sample_z(data, posterior_pi, posterior_mu, posterior_Sigma, K)
    rgb_colors_posterior = probabilities_to_rgb(probabilities_posterior, fixed_palette)
    
    plt.figure(figsize=(10, 6))
    plt.scatter(data[:, 0], data[:, 1], c=rgb_colors_posterior, marker='o')

    for k in range(K):
        plt.scatter(posterior_mu[k, 0], posterior_mu[k, 1], c='black', marker='x', s=250, linewidths=4.5)
        plt.scatter(posterior_mu[k, 0], posterior_mu[k, 1], c=[fixed_palette[k]], marker='x', s=120, linewidths=2.0)
        plot_ellipse(plt.gca(), posterior_mu[k], posterior_Sigma[k], color=fixed_palette[k])

    for k in range(K):
        plt.scatter([], [], c=[fixed_palette[k]], marker='x', s=120, label=f'Cluster {k+1} Mean')

    plt.title(title, fontsize=18)

    plt.xlabel(x_axis, fontsize=13)
    plt.ylabel(y_axis, fontsize=13)
    plt.legend(fontsize=12)


    plt.show()


In [ ]:
nu0 = 4
S0 = np.eye(2)
num_iterations = 3000
sigma_proposal = 0.001

samples_h1 = gibbs_sampling_gmm_h1(data_norm_US, K, alpha, m0, V0, S0, nu0, initialize_parameters_random_subset, sigma_proposal, num_iterations=num_iterations, h=h1)

# Get the final clustering result
z = samples_h1['z'][-1]
final_mu = samples_h1['mu'][-1]
final_Sigma = samples_h1['Sigma'][-1]


In [ ]:
plot_two_iterations(samples_h1, [350, 2990], data_norm_US, K, fixed_palette, S0, nu0, '\'Minimum Distance Maximization between Cluster Centers\' constraint (NHANES Dataset)', 'BMI', 'SBP')
plot_clusters_posterior_h(data_norm_US, samples_h1, K, fixed_palette, S0, nu0, "Posterior Means of Cluster Centers under \n\'Minimum Distance Maximization\' constraint (NHANES Dataset)", 'BMI', 'SBP', burn_in=200)

In [ ]:
nu0 = 50
S0 = 6*np.eye(2)

samples_h1_2 = gibbs_sampling_gmm_h1(data_norm_US, K, alpha, m0, V0, S0, nu0, initialize_parameters_random_subset, sigma_proposal, num_iterations=3000, h=h1)

plot_two_iterations(samples_h1_2, [15, 2800], data_norm_US, K, fixed_palette, S0, nu0, 'Incorporating Constraint h1 on NHANES dataset', 'BMI', 'SBP')
plot_clusters_posterior(data_norm_US, samples_h1_2, K, fixed_palette, S0, nu0, 'NHANES', 'BMI', 'SBP', burn_in=200)

### Constraint h2

In [ ]:
def h2(mu, sbp_ranges, bmi_ranges):
    K = len(mu)  
    blocks_assigned = set()  
    
    for k in range(K):
        bmi = mu[k, 0]  
        sbp = mu[k, 1]  
        
        block_valid = False  # To check if a valid block is found
        
        # Check SBP validity
        for sbp_idx, (sbp_low, sbp_high) in enumerate(sbp_ranges):
            if sbp_low <= sbp < sbp_high:
                # Check BMI validity
                for bmi_idx, (bmi_low, bmi_high) in enumerate(bmi_ranges):
                    if bmi_low <= bmi < bmi_high:
                        block = (sbp_idx, bmi_idx)
                        if block not in blocks_assigned:  # Ensure the block is unique
                            blocks_assigned.add(block)
                            block_valid = True
                            break  # Break out of BMI range loop if valid block is found
                if block_valid:
                    break  # Break out of SBP range loop if valid block is found
        
        if not block_valid:
            return 0  # Return 0 if no valid block is found
    
    return 1  # Return 1 if all cluster means are valid

#example to check
mu = np.array([
    [-0.6, 0.4], 
    [0.3, 1.0],  
    [0.7, 1.4]    
])

sbp_ranges = [(-np.inf, 0.5), (0.5, 1.5), (1.5, np.inf)]
bmi_ranges = [(-np.inf, -0.5), (-0.5, 0.5), (0.5, np.inf)]


validity = h2(mu, sbp_ranges, bmi_ranges)
print(validity)  


In [ ]:
# Initialize parameters with valid means
def initialize_parameters_random_subset_blocks(data, K, sbp_ranges, bmi_ranges, s=None):
    if s is not None:
        np.random.seed(s)
    N, D = data.shape
    mu = np.zeros((K, D))
    valid = False
    
    while not valid:
        random_indices = np.random.choice(N, K, replace=False)
        mu = data[random_indices]
        
        if h2(mu, sbp_ranges, bmi_ranges) == 1:
            valid = True
    
    z = np.zeros(N, dtype=int)
    for i in range(N):
        distances = np.linalg.norm(data[i] - mu, axis=1)
        z[i] = np.argmin(distances)
    
    Sigma = np.zeros((K, D, D))
    for k in range(K):
        cluster_data = data[z == k]
        if len(cluster_data) > 1:
            Sigma[k] = np.cov(cluster_data, rowvar=False)
        else:
            Sigma[k] = np.eye(D)
    
    pi = np.zeros(K)
    for k in range(K):
        pi[k] = np.sum(z == k) / N
    
    return z, pi, mu, Sigma


In [ ]:
def add_cutoff_lines(ax, sbp_cutoffs, bmi_cutoffs):
    # Add vertical lines for BMI cutoffs
    for (low, high) in bmi_cutoffs:
        if low != -np.inf:
            ax.axvline(x=low, color='green', linestyle='--')
        if high != np.inf:
            ax.axvline(x=high, color='green', linestyle='--')
    
    # Add horizontal lines for SBP cutoffs
    for (low, high) in sbp_cutoffs:
        if low != -np.inf:
            ax.axhline(y=low, color='blue', linestyle='--')
        if high != np.inf:
            ax.axhline(y=high, color='blue', linestyle='--')



def plot_two_iterations_lines(samples, iterations, data, K, fixed_palette, S0, nu0, title, x_axis, y_axis, sbp_cutoffs, bmi_cutoffs):
    fig, ax = plt.subplots(1, 2, figsize=(12, 6))  
    
    for i, iteration in enumerate(iterations):
        z = samples['z'][iteration]
        probabilities = samples['probabilities'][iteration]
        mu = samples['mu'][iteration]
        Sigma = samples['Sigma'][iteration]
        rgb_colors = probabilities_to_rgb(probabilities, fixed_palette)
        
        ax[i].scatter(data[:, 0], data[:, 1], c=rgb_colors, marker='o')
        ax[i].set_title(f'Clustered Data at Iteration {iteration} \n($\\nu_0$={nu0}, $S_0$=[[{int(S0[0,0])}, {int(S0[0,1])}]])', fontsize=15)
        ax[i].set_xlabel(x_axis, fontsize=11)
        ax[i].set_ylabel(y_axis, fontsize=11)
        
        # Add cutoff lines
        add_cutoff_lines(ax[i], sbp_cutoffs, bmi_cutoffs)
        
        # Plot the cluster means and ellipses
        for k in range(K):
            ax[i].scatter(mu[k, 0], mu[k, 1], c='black', s=250, marker='x', linewidths=4.5)
            ax[i].scatter(mu[k, 0], mu[k, 1], c=[fixed_palette[k]], s=120, marker='x', linewidths=2.0)
            plot_ellipse(ax[i], mu[k], Sigma[k], color=fixed_palette[k])
        
        # Compute the minimum distance between cluster means
        min_distance = np.inf
        for m in range(K):
            for n in range(m + 1, K):
                distance = np.linalg.norm(mu[m] - mu[n])
                min_distance = min(min_distance, distance)
        
        print(f'Minimum distance between cluster means at iteration {iteration}: {min_distance:.3f}')
        
        if i == 0:
            legend_handles = []
            for k in range(K):
                handle = plt.Line2D([0], [0], marker='x', color=fixed_palette[k], markersize=10, linestyle='None', label=f'Cluster {k} Mean')
                legend_handles.append(handle)
            ax[i].legend(handles=legend_handles, loc='upper right', prop={'size': 12})


    fig.suptitle(title, fontsize=18)
    
    plt.tight_layout(rect=[0, 0, 1, 0.95])  
    plt.show()

def plot_clusters_posterior_h_lines(data, samples, K, fixed_palette, S0, nu0, title, x_axis, y_axis, sbp_cutoffs, bmi_cutoffs, burn_in=100):
    mu_samples = np.array(samples['mu'][burn_in:])
    Sigma_samples = np.array(samples['Sigma'][burn_in:])
    pi_samples = np.array(samples['pi'][burn_in:])
    posterior_mu = np.mean(mu_samples, axis=0)
    posterior_Sigma = np.mean(Sigma_samples, axis=0)
    posterior_pi = np.mean(pi_samples, axis=0)
    
    # Print the posterior pi with cluster labels
    for k in range(K):
        print(f"Posterior pi for cluster {k+1}: {posterior_pi[k]}")

    # Calculate and print the minimum distance between cluster means
    min_distance = np.inf
    for i in range(K):
        for j in range(i+1, K):
            distance = np.linalg.norm(posterior_mu[i] - posterior_mu[j])
            min_distance = min(min_distance, distance)
    print(f"Minimum distance between cluster means: {min_distance:.3f}")

    _, probabilities_posterior = sample_z(data, posterior_pi, posterior_mu, posterior_Sigma, K)
    rgb_colors_posterior = probabilities_to_rgb(probabilities_posterior, fixed_palette)
    
    plt.figure(figsize=(10, 6))
    plt.scatter(data[:, 0], data[:, 1], c=rgb_colors_posterior, marker='o')

    # Add cutoff lines
    add_cutoff_lines(plt.gca(), sbp_cutoffs, bmi_cutoffs)

    for k in range(K):
        plt.scatter(posterior_mu[k, 0], posterior_mu[k, 1], c='black', marker='x', s=250, linewidths=4.5)
        plt.scatter(posterior_mu[k, 0], posterior_mu[k, 1], c=[fixed_palette[k]], marker='x', s=120, linewidths=2.0)
        plot_ellipse(plt.gca(), posterior_mu[k], posterior_Sigma[k], color=fixed_palette[k])

    for k in range(K):
        plt.scatter([], [], c=[fixed_palette[k]], marker='x', s=120, label=f'Cluster {k+1} Mean')

    plt.title(
    f'Posterior Means of Cluster Centers with constraint\n Incorporating Epidemiological Knowledge {title} \n'
    f'($\\nu_0$={nu0}, $S_0$=[[{int(S0[0,0])}, {int(S0[0,1])}], '
    f'[{int(S0[1,0])}, {int(S0[1,1])}]])', fontsize=18)

    plt.xlabel(x_axis, fontsize=13)
    plt.ylabel(y_axis, fontsize=13)
    plt.legend(fontsize=12)
    plt.show()


In [ ]:
def sample_mu_metropolis_h2(data, z, K, V0, m0, Sigma, mu_current, sbp_ranges, bmi_ranges, sigma_proposal, h):
    N, D = data.shape
    mu_proposed = np.zeros((K, D))
    
    # Proposal distribution (multivariate normal distribution)
    for k in range(K):
        mu_proposed[k] = multivariate_normal.rvs(mean=mu_current[k], cov=sigma_proposal * np.eye(D))
    
    # Compute the log-posterior for the current and proposed means
    log_posterior_current = 0
    log_posterior_proposed = 0
    
    for k in range(K):
        # Log-prior
        log_posterior_current += multivariate_normal.logpdf(mu_current[k], mean=m0, cov=V0)
        log_posterior_proposed += multivariate_normal.logpdf(mu_proposed[k], mean=m0, cov=V0)
        
        # Log-likelihood
        for i in range(N):
            if z[i] == k:
                log_posterior_current += multivariate_normal.logpdf(data[i], mean=mu_current[k], cov=Sigma[k])
                log_posterior_proposed += multivariate_normal.logpdf(data[i], mean=mu_proposed[k], cov=Sigma[k])
    
    # Add separation constraint term
    h_current = h(mu_current, sbp_ranges, bmi_ranges)
    h_proposed = h(mu_proposed, sbp_ranges, bmi_ranges)

    if h_current == 0:
        log_posterior_current = -np.inf
    else:
        log_posterior_current += np.log(h_current)
    
    if h_proposed == 0:
        log_posterior_proposed = -np.inf
    else:
        log_posterior_proposed += np.log(h_proposed)

    # Compute the acceptance ratio
    acceptance_ratio = log_posterior_proposed - log_posterior_current
    
    if np.isnan(acceptance_ratio):
        print(f"NaN encountered in acceptance_ratio calculation: log_posterior_proposed={log_posterior_proposed}, log_posterior_current={log_posterior_current}")


    # Accept or reject the proposal
    if np.log(np.random.uniform(0, 1)) <= acceptance_ratio:
        mu = mu_proposed
    else:
        mu = mu_current
    
    return mu


In [ ]:
def gibbs_sampling_gmm_h2(data, K, alpha, m0, V0, S0, nu0, initialization_function, sbp_ranges, bmi_ranges, sigma_proposal, num_iterations, h, s=None):
    N, D = data.shape
    
    # Initialize variables

    z, pi, mu, Sigma = initialization_function(data, K, sbp_ranges, bmi_ranges, s)
    
    samples = {'z': [], 'probabilities': [], 'pi': [], 'mu': [], 'Sigma': [], 'log_likelihood': []}
    
    for iteration in range(num_iterations):
        # Sample z
        z, probabilities = sample_z(data, pi, mu, Sigma, K)
        
        # Sample pi
        pi = sample_pi(z, K, alpha)
        
        # Sample mu using Metropolis within Gibbs
        mu = sample_mu_metropolis_h2(data, z, K, V0, m0, Sigma, mu, sbp_ranges, bmi_ranges, sigma_proposal, h)
        
        # Sample Sigma
        Sigma = sample_Sigma(data, z, K, mu, S0, nu0)
        
        # Store log-likelihood
        log_likelihood = compute_log_likelihood(data, pi, mu, Sigma, K)
        samples['log_likelihood'].append(log_likelihood)
        
        # Append current samples to the list
        samples['z'].append(z.copy())
        samples['probabilities'].append(probabilities.copy())
        samples['pi'].append(pi.copy())
        samples['mu'].append(mu.copy())
        samples['Sigma'].append(Sigma.copy())
    
    return samples


In [ ]:
mean_sbp = mean_US[1]
std_sbp = std_US[1]
mean_bmi = mean_US[0]
std_bmi = std_US[0]

# Function to normalize cutoffs
def normalize_cutoffs(cutoffs, mean, std):
    normalized_cutoffs = [( (low - mean) / std, (high - mean) / std if high != np.inf else np.inf) for (low, high) in cutoffs]
    return normalized_cutoffs

# Cutoff ranges for SBP and BMI
sbp_cutoff = [(float('-inf'), 140), (140, 160), (160, float('inf'))]
bmi_cutoff = [(float('-inf'),18.5),(18.5, 25), (25, 30), (30, float('inf'))]
sbp_cutoffs = normalize_cutoffs(sbp_cutoff, mean_sbp, std_sbp)
bmi_cutoffs = normalize_cutoffs(bmi_cutoff, mean_bmi, std_bmi)


z, pi, mu, Sigma = initialize_parameters_random_subset_blocks(data_norm_US, K, sbp_cutoffs, bmi_cutoffs, s=452)


# Plot the initial cluster means and cutoff lines
plt.figure(figsize=(10, 6))
plt.scatter(data_norm_US[:, 0], data_norm_US[:, 1], c='gray', alpha=0.5, label='Data points')
plt.scatter(mu[:, 0], mu[:, 1], c='red', marker='x', s=100, label='Cluster means')

# Plot cutoff lines for BMI
for low, high in bmi_cutoffs:
    if high != np.inf:
        plt.axvline(x=low, linestyle='--', color='green')
        plt.axvline(x=high, linestyle='--', color='green')
    else:
        plt.axvline(x=low, linestyle='--', color='green')

# Plot cutoff lines for SBP
for low, high in sbp_cutoffs:
    if high != np.inf:
        plt.axhline(y=low, linestyle='--', color='blue')
        plt.axhline(y=high, linestyle='--', color='blue')
    else:
        plt.axhline(y=low, linestyle='--', color='blue')

plt.xlabel('Normalized BMI')
plt.ylabel('Normalized SBP')
plt.legend()
plt.title('Initial Cluster Means and Cutoff Ranges')
plt.show()


In [ ]:
# Run Gibbs sampling on US data with h2 
S0 = np.eye(2)
nu0 = 4
sigma_proposal=0.001
samples_blocks = gibbs_sampling_gmm_h2(data_norm_US, K, alpha, m0, V0, S0, nu0, initialize_parameters_random_subset_blocks, sbp_ranges=sbp_cutoffs, bmi_ranges=bmi_cutoffs, sigma_proposal=sigma_proposal, num_iterations=3000, h=h2)


In [ ]:
plot_two_iterations_lines(samples_blocks, [1300, 2000], data_norm_US, K, fixed_palette, S0, nu0, 'Incorporating Constraint with Epidemiological Knowledge (NHANES Dataset)\n'f'($\\nu_0$={nu0}, $S_0$=[[{int(S0[0,0])}, {int(S0[0,1])}], '
    f'[{int(S0[1,0])}, {int(S0[1,1])}]])', 'BMI', 'SBP', sbp_cutoffs, bmi_cutoffs)
plot_clusters_posterior_h_lines(data_norm_US, samples_blocks, K, fixed_palette, S0, nu0, '(NHANES Dataset)', 'BMI', 'SBP', sbp_cutoffs, bmi_cutoffs, burn_in=200)


In [ ]:
S0 = 6* np.eye(2)
nu0 = 50

samples_blocks_2 = gibbs_sampling_gmm_h2(data_norm_US, K, alpha, m0, V0, S0 , nu0, initialize_parameters_random_subset_blocks, sbp_ranges=sbp_cutoffs, bmi_ranges=bmi_cutoffs, sigma_proposal=sigma_proposal, num_iterations=3000, h=h2)

In [ ]:
plot_two_iterations_lines(samples_blocks_2, [700, 2950], data_norm_US, K, fixed_palette, S0, nu0, 'Incorporating Constraint with Epidemiological Knowledge (NHANES Dataset)\n'f'($\\nu_0$={nu0}, $S_0$=[[{int(S0[0,0])}, {int(S0[0,1])}], '
    f'[{int(S0[1,0])}, {int(S0[1,1])}]])', 'BMI', 'SBP', sbp_cutoffs, bmi_cutoffs)
plot_clusters_posterior_h_lines(data_norm_US, samples_blocks_2, K, fixed_palette, S0, nu0, '(NHANES Dataset)', 'BMI', 'SBP', sbp_cutoffs, bmi_cutoffs, burn_in=100)

### Higher dimension

In [ ]:
import seaborn as sns


In [ ]:
# Load and prepare the data (adjusted to include more variables)
df = pd.read_csv('NHANES_adults_data_preproc.csv')
variables = ['bmi', 'sbp', 'non_hdl','hba1c']  
df_variables = df[variables]

# Check for missing values in the selected columns
missing_values = df_variables.isnull().sum()
print("Missing values before removal:")
print(missing_values)

df_variables_clean = df_variables.dropna()

# Normalize the entire dataset
data_US = df_variables_clean.values
data_norm_US_full, mean_US, std_US = normalize_data(data_US)

data_norm_US_full_df = pd.DataFrame(data_norm_US_full, columns=variables)

# Take a random subset of the normalized data
np.random.seed(5)
data_norm_US = data_norm_US_full_df.sample(n=300, replace=False, random_state=5)
data_norm_US


data_norm_US = data_norm_US.values
data_norm_US

In [ ]:
# Trace plots
def plot_trace_plots(samples, num_iterations, K, S0, nu0):
    fig, axes = plt.subplots(3, 1, figsize=(8, 14))

    # Plot trace of pi
    for k in range(K):
        axes[0].plot([samples['pi'][i][k] for i in range(num_iterations)], label=f'pi_{k}')
    axes[0].set_title(f'Trace plot of pi with S0={S0}, nu0={nu0}')
    axes[0].legend()

    # Plot trace of mu
    for k in range(K):
        for d in range(samples['mu'][0].shape[1]):
            axes[1].plot([samples['mu'][i][k, d] for i in range(num_iterations)], label=f'mu_{k}_{d}')
    axes[1].set_title(f'Trace plot of mu with S0={S0}, nu0={nu0}')
    axes[1].legend()

    # Plot trace of Sigma (diagonal elements only)
    for k in range(K):
        for d in range(samples['Sigma'][0].shape[1]):
            axes[2].plot([samples['Sigma'][i][k, d, d] for i in range(num_iterations)], label=f'Sigma_{k}_{d}_{d}')
    axes[2].set_title(f'Trace plot of Sigma (diagonal elements) with S0={S0}, nu0={nu0}')
    axes[2].legend()

    plt.tight_layout()
    plt.show()

In [ ]:
# Update to handle the constraints for multiple variables
def initialize_parameters_random_subset_blocks_multid(data, K, variable_ranges, s=None):
    if s is not None:
        np.random.seed(s)
    N, D = data.shape
    mu = np.zeros((K, D))
    valid = False
    
    while not valid:
        random_indices = np.random.choice(N, K, replace=False)
        mu = data[random_indices]
        
        if h2_multid(mu, variable_ranges) == 1:
            valid = True
    
    z = np.zeros(N, dtype=int)
    for i in range(N):
        distances = np.linalg.norm(data[i] - mu, axis=1)
        z[i] = np.argmin(distances)
    
    Sigma = np.zeros((K, D, D))
    for k in range(K):
        cluster_data = data[z == k]
        if len(cluster_data) > 1:
            Sigma[k] = np.cov(cluster_data, rowvar=False)
        else:
            Sigma[k] = np.eye(D)
    
    pi = np.zeros(K)
    for k in range(K):
        pi[k] = np.sum(z == k) / N
    
    return z, pi, mu, Sigma


def h2_multid(mu, ranges):
    K = len(mu)  # Number of clusters
    num_vars = mu.shape[1]  # Number of variables (dimensions)
    

    unique_combinations = set()
    
    for k in range(K):
        combination = []
        
        for dim in range(num_vars):
            value = mu[k, dim]
            dimension_ranges = ranges[dim]
            range_index = None
            
    
            for idx, (low, high) in enumerate(dimension_ranges):
                if low <= value < high:
                    range_index = idx
                    break
            
            if range_index is None:
                print(f"Value {value} for dimension {dim} does not fall into any range.")
                return 0  # Return 0 if the value doesn't fall into any range
            
            combination.append(range_index)
        
        combination_tuple = tuple(combination)
        
  
        
        unique_combinations.add(combination_tuple)
    
    return 1 

# Update to handle multiple variables
def sample_mu_metropolis_h2_multid(data, z, K, V0, m0, Sigma, mu_current, variable_ranges, sigma_proposal, h):
    N, D = data.shape
    mu_proposed = np.zeros((K, D))
    
    for k in range(K):
        mu_proposed[k] = multivariate_normal.rvs(mean=mu_current[k], cov=sigma_proposal * np.eye(D))
    
    log_posterior_current = 0
    log_posterior_proposed = 0
    
    for k in range(K):
        log_posterior_current += multivariate_normal.logpdf(mu_current[k], mean=m0, cov=V0)
        log_posterior_proposed += multivariate_normal.logpdf(mu_proposed[k], mean=m0, cov=V0)
        
        for i in range(N):
            if z[i] == k:
                log_posterior_current += multivariate_normal.logpdf(data[i], mean=mu_current[k], cov=Sigma[k])
                log_posterior_proposed += multivariate_normal.logpdf(data[i], mean=mu_proposed[k], cov=Sigma[k])
    
    h_current = h(mu_current, variable_ranges)
    h_proposed = h(mu_proposed, variable_ranges)

    if h_current == 0:
        log_posterior_current = -np.inf
    else:
        log_posterior_current += np.log(h_current)
    
    if h_proposed == 0:
        log_posterior_proposed = -np.inf
    else:
        log_posterior_proposed += np.log(h_proposed)

    acceptance_ratio = log_posterior_proposed - log_posterior_current
    
    if np.isnan(acceptance_ratio):
        print(f"NaN encountered in acceptance_ratio calculation: log_posterior_proposed={log_posterior_proposed}, log_posterior_current={log_posterior_current}")

    if np.log(np.random.uniform(0, 1)) <= acceptance_ratio:
        mu = mu_proposed
    else:
        mu = mu_current
    
    return mu



def gibbs_sampling_gmm_h2_multid(data, K, alpha, m0, V0, S0, nu0, initialization_function, variable_ranges, sigma_proposal, num_iterations, h, s=None):
    N, D = data.shape
    
    # Initialize variables
    z, pi, mu, Sigma = initialization_function(data, K, variable_ranges, s)
    
    samples = {'z': [], 'probabilities': [], 'pi': [], 'mu': [], 'Sigma': [], 'log_likelihood': []}
    
    for iteration in range(num_iterations):
        # Sample z
        z, probabilities = sample_z(data, pi, mu, Sigma, K)
        
        # Sample pi
        pi = sample_pi(z, K, alpha)
        
        # Sample mu using Metropolis within Gibbs
        mu = sample_mu_metropolis_h2_multid(data, z, K, V0, m0, Sigma, mu, variable_ranges, sigma_proposal, h)
        
        # Sample Sigma
        Sigma = sample_Sigma(data, z, K, mu, S0, nu0)
        
        # Store log-likelihood
        log_likelihood = compute_log_likelihood(data, pi, mu, Sigma, K)
        samples['log_likelihood'].append(log_likelihood)
        
        # Append current samples to the list
        samples['z'].append(z.copy())
        samples['probabilities'].append(probabilities.copy())
        samples['pi'].append(pi.copy())
        samples['mu'].append(mu.copy())
        samples['Sigma'].append(Sigma.copy())
    
    return samples



In [ ]:
# Define cutoff ranges for the selected variables
cutoff_ranges = {
    'bmi': [(float('-inf'),18.5),(18.5, 25), (25, 30), (30, float('inf'))],
    'sbp': [(float('-inf'), 140), (140, 160), (160, float('inf'))],  
    'non_hdl': [(float('-inf'), 3.36), (3.36, 5.69), (5.69, float('inf'))], 
    'hba1c': [(float('-inf'), 6.5), (6.5, 10.0), (10.0, float('inf'))],
}


mean_std_values = {
    'bmi': (mean_US[0], std_US[0]),
    'sbp': (mean_US[1], std_US[1]),  
    'non_hdl': (mean_US[2], std_US[2]),  
    'hba1c': (mean_US[3], std_US[3]), 
}

# Apply normalization to the cutoff ranges
normalized_cutoff_ranges = {var: normalize_cutoffs(cutoff, mean_std_values[var][0], mean_std_values[var][1]) 
                            for var, cutoff in cutoff_ranges.items()}

# Convert to list of ranges for the Gibbs sampling function
variable_ranges = list(normalized_cutoff_ranges.values())

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import gaussian_kde

def plot_posterior_distributions(samples, variables, K, population_distributions, title, burn_in=0, mean_std_values=None, cutoff_ranges=None):

    mu_samples = samples['mu'][burn_in:]

    num_iterations = len(mu_samples)
    num_variables = len(variables)

    # Define color palette for clusters
    colors = sns.color_palette("husl", K)

    fig, axes = plt.subplots(2, 2, figsize=(14, 10))  
    fig.suptitle(title, fontsize=20)


    x_labels = {
        'bmi': 'BMI (Body Mass Index)',
        'sbp': 'SBP (Blood Systolic Pressure)',
        'non_hdl': 'Non-HDL (Non-high-density lipoprotein cholesterol)',
        'hba1c': 'HbA1c (Glycated Hemoglobin)'
    }

    labels = {
        'bmi': 'BMI',
        'sbp': 'SBP',
        'non_hdl': 'Non-HDL',
        'hba1c': 'HbA1c'
    }

    for var_idx, var_name in enumerate(variables):
 
        row, col = divmod(var_idx, 2)
        ax = axes[row, col]
        
        for k in range(K):
            means = [mu_samples[i][k, var_idx] for i in range(num_iterations)]
            
            # Transform the means back to original scale
            original_means = [(mean_std_values[var_name][0] + m * mean_std_values[var_name][1]) for m in means]
            
            sns.kdeplot(original_means, ax=ax, color=colors[k], lw=2, label=f'Cluster {k+1}')

        x = np.linspace(ax.get_xlim()[0], ax.get_xlim()[1], 100)
        population_density = population_distributions[var_idx](x)
        ax.plot(x, population_density, color='red', linestyle='--', alpha=0.5, 
                label=f'Population Distribution for {labels[var_name]}')

        ax.set_xlabel(x_labels.get(var_name, var_name), fontsize=12)  
        ax.set_ylabel('Density',fontsize=12)
        ax.legend()
 
        if cutoff_ranges and var_name in cutoff_ranges:
            for (low, high) in cutoff_ranges[var_name]:
                if low != -np.inf:
                    original_low = mean_std_values[var_name][0] + low * mean_std_values[var_name][1]
                    ax.axvline(original_low, color='black', linestyle='--', alpha=0.7)
                if high != np.inf:
                    original_high = mean_std_values[var_name][0] + high * mean_std_values[var_name][1]
                    ax.axvline(original_high, color='black', linestyle='--', alpha=0.7)

    plt.tight_layout(rect=[0, 0, 1, 1])
    plt.show()


population_distributions = []
for var_idx in range(data_US.shape[1]):
    kde = gaussian_kde(data_US[:, var_idx])
    population_distributions.append(kde)

In [ ]:
def compute_posterior_means(samples_h2, burn_in):
    num_iterations_after_burnin = len(samples_h2['mu']) - burn_in
    mu_samples = np.array(samples_h2['mu'][burn_in:])
    pi_samples = np.array(samples_h2['pi'][burn_in:])
    Sigma_samples = np.array(samples_h2['Sigma'][burn_in:])
    # Compute posterior mean for mu
    posterior_mean_mu = np.mean(mu_samples, axis=0)
    # Compute posterior mean for pi
    posterior_mean_pi = np.mean(pi_samples, axis=0)
    # Compute posterior mean for Sigma
    posterior_mean_Sigma = np.mean(Sigma_samples, axis=0)
    return posterior_mean_mu, posterior_mean_pi, posterior_mean_Sigma


In [ ]:
# Parameters for the Gibbs sampling
K = 5
alpha = np.ones(K)
m0 = np.zeros(len(variables)) 
V0 = np.eye(len(variables))
nu0 = len(variables) + 2
sigma_proposal = 0.001
S0 =  np.eye(len(variables))


In [ ]:
samples_standard = gibbs_sampling_gmm(
    data_norm_US, 
    K, 
    alpha, 
    m0, 
    V0, 
    S0, 
    nu0, 
    initialize_parameters_random_subset,
    num_iterations=3500
)


In [ ]:

posterior_mean_mu_standard, posterior_mean_pi_standard, posterior_mean_Sigma_standard = compute_posterior_means(samples_standard,200)
print("\nPosterior Mean of pi:")


In [ ]:
plot_posterior_distributions(
    samples=samples_standard,  
    variables=variables,
    K=K,
    population_distributions=population_distributions,
    title='Posterior Distributions of Cluster Centers Across Variables',
    burn_in=200,
    mean_std_values=mean_std_values,
    cutoff_ranges=normalized_cutoff_ranges,
)

In [ ]:
samples_h1 = gibbs_sampling_gmm_h1(
    data_norm_US, 
    K, 
    alpha, 
    m0, 
    V0, 
    S0, 
    nu0, 
    initialize_parameters_random_subset,  # Initialization without constraints
    sigma_proposal, 
    num_iterations=3500, 
    h=h1
)

In [ ]:

posterior_mean_mu_h1, posterior_mean_pi_h1, posterior_mean_Sigma_h1 = compute_posterior_means(samples_h1,200)
print("\nPosterior Mean of pi:")
print(posterior_mean_pi_h1)

plot_posterior_distributions(
    samples=samples_h1,  
    variables=variables,
    K=K,
    population_distributions=population_distributions,
    title='Posterior Distributions of Cluster Centers Across Variables \n(GMM with Minimum Distance Maximization Constraint)',
    burn_in=200,
    mean_std_values=mean_std_values,
    cutoff_ranges=normalized_cutoff_ranges
)


In [ ]:

samples_h2 = gibbs_sampling_gmm_h2_multid(
    data_norm_US, 
    K, 
    alpha, 
    m0, 
    V0, 
    S0, 
    nu0, 
    initialize_parameters_random_subset_blocks_multid, 
    variable_ranges, 
    sigma_proposal, 
    num_iterations=3500, 
    h=h2_multid
)

In [ ]:

posterior_mean_mu_h2, posterior_mean_pi_h2, posterior_mean_Sigma_h2 = compute_posterior_means(samples_h2,200)
print("\nPosterior Mean of pi:")
print(posterior_mean_pi_h2)

plot_posterior_distributions(
    samples=samples_h2, 
    variables=variables,
    K=K,
    population_distributions=population_distributions,
    title='Posterior Distributions of Cluster Centers Across Variables \n(GMM Incorporating Epidemiological Knowledge)',
    burn_in=200,
    mean_std_values=mean_std_values,
    cutoff_ranges=normalized_cutoff_ranges,

)

